<a href="https://colab.research.google.com/github/arkalim/Deep-Learning/blob/master/Cat_vs_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
###### Mount Google Drive #######

from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Import Libraries

from keras.applications.xception import Xception
from keras.models import Sequential
from keras.layers.core import Dense , Flatten , Dropout
from keras.optimizers import Adam ,RMSprop
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from keras.layers import Input
from keras.models import Model
import cv2
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib.image as img
from keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau
from keras import layers

In [0]:
#Load and Preprocess Data

image_dim = 256
set_trainable = False

train_path = r"/content/drive/My Drive/AI/DL/Cats vs Dogs/Train"
valid_path = r"/content/drive/My Drive/AI/DL/Cats vs Dogs/Valid"
test_path =  r"/content/drive/My Drive/AI/DL/Cats vs Dogs/Test"

train_datagen = ImageDataGenerator( rescale=1./255,
                                    rotation_range=40,
                                    shear_range=0.3,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

valid_datagen = ImageDataGenerator( rescale=1./255,
                                    rotation_range=40,
                                    shear_range=0.3,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator( rescale=1./255)

train_batches = train_datagen.flow_from_directory(train_path, target_size=(image_dim,image_dim), classes=['Cat', 'Dog'], batch_size = 10)
valid_batches = valid_datagen.flow_from_directory(valid_path, target_size=(image_dim,image_dim), classes=['Cat', 'Dog'], batch_size = 10)
test_batches = test_datagen.flow_from_directory(test_path, target_size=(image_dim,image_dim), classes=['Cat', 'Dog'], batch_size = 1)



In [0]:
#Load the Model

conv_base = Xception(weights='imagenet', include_top=False, input_shape=(image_dim,image_dim,3))

model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

conv_base.trainable = True

'''
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
'''
model.summary() 


In [0]:
#Compile and Train the Model

callbacks_list = [ ModelCheckpoint(filepath="/content/drive/My Drive/AI/DL/Cats vs Dogs/Trained_Model_256.h5", monitor='val_loss', save_best_only=True), ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=3) ]

model.compile(Adam(lr = 0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])  

history = model.fit_generator(train_batches, steps_per_epoch=300, validation_data = valid_batches, callbacks=callbacks_list, validation_steps=100, epochs =40)

In [0]:
#Predict using the trained model

image = img.imread(r"/content/drive/My Drive/AI/DL/test1/6.jpg")

print('saving the model')
model.save(r"/content/drive/My Drive/AI/DL/Cats vs Dogs/Trained_Model.h5")
print('model saved')

image = cv2.resize(image , (image_dim,image_dim))
image = image.astype('float32')
image = image/255
image = np.expand_dims(image, axis=0)

prediction = model.predict(image)
print(np.argmax(prediction))


In [0]:
################################################################################
#Printing the accuracy and loss curves

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color = 'b',label='Training acc')
plt.plot(epochs, val_acc, color = 'g', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,color = 'b', label='Training loss')
plt.plot(epochs, val_loss, color = 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

################################################################################    